## 1、构造向量数据库

In [1]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import QianfanEmbeddingsEndpoint # 调用 百度文心一言 的 Embeddings 模型
from dotenv import load_dotenv, find_dotenv
import os
# _ = load_dotenv(find_dotenv()) # read local .env file
_ = load_dotenv('../QA_Project/.env')

# 获取环境变量 
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]


# 定义 Embeddings
qianfan_embedding = QianfanEmbeddingsEndpoint(qianfan_ak=wenxin_api_key,
                                  qianfan_sk=wenxin_secret_key) 

# 向量数据库持久化路径
persist_directory = '../QA_Project/data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=qianfan_embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：2196


In [2]:
from langchain.prompts import PromptTemplate

template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)

from langchain.chains import RetrievalQA
from langchain_community.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint

llm = QianfanChatEndpoint(
    qianfan_ak=wenxin_api_key,
    qianfan_sk=wenxin_secret_key,model='ERNIE-Bot-4'
)

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [101]:
question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 14:55:55] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 14:55:55] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/completions_pro


南瓜书是一本名为《机器学习公式详解》的书，它对西瓜书的内容进行了深入的阐述和补充，特别是那些难以理解的公式部分。南瓜书的设计初衷是作为西瓜书的伴侣，帮助读者更好地理解和掌握机器学习的知识。谢谢你的提问！


In [9]:
result

{'query': '我们应该如何去构造一个LLM项目',
 'result': '为了构造一个LLM项目，我们可以按照以下步骤进行：\n\n1. 确定项目目标和需求：在开始构造LLM项目之前，我们需要明确项目的目标和需求。这包括确定项目要解决的具体问题、应用场景以及所需的功能。通过明确目标和需求，我们可以为项目的后续工作提供指导。\n2. 选择适合的LLM类型和模型：根据项目需求和目标，我们可以选择适合的LLM类型和模型。基础LLM适用于一般性的文本生成任务，而指令微调（Instruction Tuned）LLM则更适合对指令高度敏感的任务。此外，还可以根据具体需求选择预训练模型或自定义模型。\n3. 设计Prompt：Prompt是引导LLM生成结果的输入。设计一个好的Prompt对于LLM项目的成功至关重要。Prompt应该简洁明了，能够准确表达我们的意图和需求。在设计Prompt时，我们可以参考本书下一章关于提示词设计的两个关键原则：清晰明确和给予充足思考时间。\n4. 收集和准备数据：根据项目需求和所选的LLM模型，我们需要收集和准备相应的数据。这包括训练数据、验证数据和测试数据。数据的质量和数量对于LLM模型的训练效果和性能有着重要影响。\n5. 训练和调整模型：使用收集到的数据进行模型训练。在训练过程中，我们可以根据需要对模型进行调整，以优化其性能。例如，可以通过调整模型参数、改变训练策略或应用正则化技术来提高模型的泛化能力。\n6. 评估模型性能：训练完成后，我们需要对模型进行评估，以确保其性能满足项目需求。评估可以包括定性和定量两个方面。定性评估可以通过人工检查生成的结果来判断模型的质量，而定量评估则可以使用指标如准确率、召回率等来衡量模型的性能。\n7. 部署和监控模型：一旦模型性能满足要求，我们可以将其部署到实际应用中。在部署后，我们需要持续监控模型的性能，以确保其在实际应用中的稳定性和可靠性。如果发现问题或性能下降，我们可以及时调整模型或重新训练。\n\n通过以上步骤，我们可以构造一个有效的LLM项目，并利用LLM的能力来加速工作进程和提高工作效率。同时，我们也需要注意不断学习和更新知识库，以便不断优化和完善我们的LLM项目。',
 'source_documents': [Document(page_content='如果你将 LLM 视为

## 2、提升直观回答质量

In [10]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v2)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])


[INFO] [03-12 15:54:20] openapi_requestor.py:316 [t:139785230325568]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 15:54:20] openapi_requestor.py:316 [t:139785230325568]: requesting llm api endpoint: /chat/eb-instant


南瓜书《机器学习公式详解》的名称来源于书中对机器学习算法中常见公式的解释和推导过程。它不同于西瓜书，南瓜书更侧重于公式的推导和详解，适合对机器学习算法有深入理解和应用需求的读者。

具体来说，南瓜书的内容是以西瓜书的内容为前置知识进行表述的，因此读者在阅读南瓜书之前，需要先掌握西瓜书的相关内容。南瓜书中的公式推导过程较为详细，对于初学机器学习的读者来说，第 1 章和第 2 章的公式可能较为复杂，不建议深究，简单过一下即可。而对于有一定基础的读者来说，南瓜书中的公式推导过程可以帮助他们更好地理解和掌握机器学习算法的原理和应用。

总的来说，南瓜书《机器学习公式详解》是一本针对机器学习爱好者和专业人士的书籍，适合那些对机器学习有深入理解和应用需求的读者。


In [22]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 10:31:21] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 10:31:21] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/eb-instant


使用大模型时，构造 Prompt 的原则主要有两个：明确性和给模型思考时间。

明确性是指 Prompt 的指令要清晰、具体，让模型能够明确理解需要完成的任务。这包括使用简洁、明确的语言，避免使用模糊或歧义的词汇，以及确保 Prompt 的结构符合语言模型的训练数据格式。

给模型思考时间则是为了让语言模型有充足的时间进行推理和生成。匆忙得出的结论往往不够准确，因此 Prompt 中应该加入逐步推理的要求，给模型留出充分思考的时间，这样生成的结果才更准确可靠。

在迭代优化过程中，设计高效 Prompt 需要多次尝试和调整。第一版 Prompt 应该满足明确性和给模型思考时间这两个原则，在此基础上，一般需要通过多轮迭代逐步逼近最优。许多成功的 Prompt 都是通过多次试错调整得出的。


In [3]:
template_v3 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v3)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

/root/miniconda3/envs/aleienv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
[INFO] [03-12 15:57:42] openapi_requestor.py:316 [t:139945379809088]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 15:57:42] oauth.py:207 [t:139945379809088]: trying to refresh access_token for ak `NSwW3Z***`
[INFO] [03-12 15:57:42] oauth.py:220 [t:139945379809088]: sucessfully refresh access_token
[INFO] [03-12 15:57:42] openapi_requestor.py:316 [t:139945379809088]: requesting llm api endpoint: /chat/completions_pro


使用大模型时，构造Prompt的原则主要包括以下两点：

1. 编写清晰、具体的指令：这一点是为了确保大模型能够准确理解任务需求。通过给出明确、具体的指令，可以避免模型产生歧义或误解，从而提高其生成结果的准确性和可靠性。
2. 给予模型充足思考时间：这一点是为了让大模型有足够的时间进行推理和生成。与人类解题类似，如果过于匆忙地得出结论，很可能会导致失误。因此，在构造Prompt时，应加入逐步推理的要求，并为模型留出充分的思考时间，以确保其生成的结果更准确、更可靠。

掌握这些Prompt设计原则，对于开发者来说，是取得语言模型应用成功的重要一步。同时，良好的迭代优化过程也是不断改进Prompt、找到最适合应用形式的关键。


In [26]:
question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 10:45:59] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 10:45:59] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/completions_pro


强化学习是一种机器学习的方法，它通过与环境的交互来学习如何做出一系列好的决策，以最大化累积奖励。在强化学习中，智能体根据当前环境状态选择并执行一个动作，环境会根据该动作给出新的状态和奖励，智能体再根据新的状态和奖励进行下一步的决策，如此往复。谢谢你的提问！


In [32]:
template_v4 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
请你附上回答的来源原文，以保证回答的正确性。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 10:51:11] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 10:51:11] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/completions_pro


强化学习是一种机器学习方法，其主要目标是让智能体（agent）在不确定的环境中学习如何做出一系列好的决策，以最大化累积奖励。强化学习的特点在于，它不需要预先给出所有可能的场景和对应的最优策略，而是通过与环境的交互，不断地试错并调整策略，最终达到最优决策的目的。在强化学习过程中，智能体会接收到环境的状态信息，并根据当前策略选择一个动作执行。执行动作后，环境会给出新的状态以及奖励，智能体根据这些信息来更新自己的策略和价值函数，以便在将来的决策中更好地选择动作。

由于强化学习具有自学习和自适应的能力，因此在很多领域都有广泛的应用，例如游戏、自动驾驶、机器人控制、医疗决策等。通过学习强化学习，可以帮助我们更好地理解智能决策的本质，为实际应用提供更为高效和智能的解决方案。

需要注意的是，以上仅为强化学习的简要介绍，强化学习的具体实现和算法可能因不同的任务和场景而有所不同。如果您想深入了解强化学习的原理和应用，请参考相关的教材和资料。

（该段回答参考了多个来源和教材，由于篇幅限制，未能列出具体的参考文献。如有需要，请在进一步的学习和研究中查阅相关文献。）


In [12]:
question = "如何使用gradio搭建前端页面"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 16:43:51] openapi_requestor.py:316 [t:140204924962624]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 16:43:51] openapi_requestor.py:316 [t:140204924962624]: requesting llm api endpoint: /chat/completions_pro


很抱歉，您提供的上下文中并未涉及关于如何使用gradio搭建前端页面的信息，而且gradio库在上述代码中并未提及。gradio是一个用于快速创建界面的Python库，可以方便地为机器学习模型等创建交互式界面，但并不是上述代码中的一部分。

在一般的情况下，如果您想要使用gradio搭建前端页面，通常需要以下几个步骤：

1. 安装gradio库，这可以通过Python的包管理工具pip完成。

2. 定义您希望展示的函数，这可以是一个机器学习模型，也可以是其他任何可以通过输入参数得到输出的Python函数。

3. 使用gradio.Interface或其他gradio组件创建交互式界面，您需要指定输入和输出的类型，以及可能的其他参数。

然而，这个答案并未基于您提供的上下文得出，因为您的上下文中并未涉及gradio或相关的内容。所以，我不能给出与您上下文紧密相关的答案。如果您能提供更多关于如何在您特定的代码或项目中使用gradio的信息，我可能能够提供更准确的帮助。

请注意，以上提供的信息是通用的，可能与您的具体情况有所不同，具体情况可能因您所使用的环境、版本或其他因素而有所不同。建议您查看gradio的官方文档或其他可靠的学习资源以获取最准确和最新的信息。


In [11]:
template_v4 = """
请按照以下步骤回答问题：

1. 仔细阅读以下上下文：
{context}

2. 基于上述上下文，回答以下问题：
{question}

3. 回答时请注意：
   - 不要添加任何不在上下文中的信息，不要给出一般性的回答或尝试根据一般知识来回答。
   - 如果答案较长，请分点陈述。
   - 确保回答与上下文内容紧密相关，不偏离主题。
   -  基于提供的上下文，反思回答中有没有不正确或不是基于上下文得到的内容，如果有，修改回答。

请给出您的回答。
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "如何使用gradio搭建前端页面"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 16:37:25] openapi_requestor.py:316 [t:140204924962624]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 16:37:26] openapi_requestor.py:316 [t:140204924962624]: requesting llm api endpoint: /chat/completions_pro


很抱歉，根据您提供的上下文，关于“如何使用gradio搭建前端页面”的问题并没有直接的信息。上下文主要讨论了关于一个点餐机器人的实现和一个JSON摘要的创建要求，但没有具体涉及到gradio库的使用或前端页面的搭建。

然而，我可以根据一般知识提供一些关于如何使用gradio搭建前端页面的基本步骤，但请注意，这些步骤可能与您提供的上下文不完全相关：

1. **安装gradio库**：首先，您需要安装gradio库。您可以使用pip命令进行安装：`pip install gradio`。
2. **定义接口**：在gradio中，您需要定义一个接口（Interface），该接口描述了您的模型的输入和输出。您可以使用gradio提供的各种输入和输出组件来定义接口。
3. **创建前端页面**：使用gradio的`Interface`类创建一个前端页面。您可以设置页面的标题、描述、输入和输出组件等。
4. **启动服务**：使用gradio的`launch`方法启动一个本地服务，该服务将托管您创建的前端页面，并允许用户与您的模型进行交互。

请注意，这些步骤只是一般性的指导，具体的实现可能会根据您的具体需求和上下文而有所不同。如果您需要更详细的指导或有关如何在特定上下文中使用gradio的具体信息，请提供更多的上下文或相关代码示例。

另外，我注意到在提供的上下文中存在一些与问题无关的信息，如“MATERIALS”和“COUNTRY OF ORIGIN”等部分，这些信息似乎与如何使用gradio搭建前端页面没有直接关系。如果这些信息与问题无关，请忽略它们。


In [90]:
question = "LLM的分类是什么？给我返回一个 Python List，注意，只返回列表，不要markdown格式。"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [03-12 14:12:03] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 14:12:03] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/completions_pro


1. 首先，关于"LLM的分类"，在上下文中并未明确给出LLM是什么，也没有关于其分类的直接信息。因此，我不能基于提供的上下文来回答LLM的分类。但是，假设LLM是某种对象或实体的缩写，并且我们需要对其进行分类，通常我们会需要一个明确的定义或描述来确定其分类。不过，在此情况下，我不能提供一个准确的分类列表。

对于要求返回一个Python List的要求，由于上下文中没有提供足够的信息来确定LLM的分类，所以我无法直接返回一个列表。但为了满足格式要求，我可以展示一个空的Python列表或示例列表的格式。例如：


```python
llm_categories = []
```
或者，如果有分类信息的话，它可能看起来像这样：


```python
llm_categories = ['Category 1', 'Category 2', 'Category 3']
```
但请注意，上面的分类只是示例，并不代表任何实际的LLM分类。

2. 基于提供的上下文，我注意到以下几点可能需要修正或澄清：


	* 在展示HTML代码的部分，使用了`display(HTML(response))`，但在提供的上下文中，`response` 的内容并没有给出，它应该是从 `get_completion(prompt)` 得到的。这部分逻辑上没有问题，但在实际执行时，需要确保 `get_completion` 函数和 `response` 变量已经正确定义。
	* 在“2.1 商品信息提取”部分，`prompt` 字符串中引用了 `{prod_review}`，但上下文中并没有给出 `prod_review` 的内容。在实际执行这段代码之前，需要确保 `prod_review` 已经被定义并包含了要分析的评论文本。
	* 同样，在“2.1 同时概括多条文本”部分，`review_1 = prod_review` 这行代码表明 `prod_review` 应该是一个已经定义的变量，但在提供的上下文中并没有给出它的内容。
	* 最后的代码块中，`prompt` 字符串尝试从 `data_json` 创建一个HTML表格，但 `data_json` 的内容也没有在上下文中给出。

综上所述，要修正回答，我们需要确保所有引用的变量（如 `response`, `prod_review

In [48]:
import requests
import json

def get_access_token():
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """
    # 指定网址
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=NSwW3Zr7f1rW8rRByKyqPKOV&client_secret=rbt3kGXuophuE8XOm45A5M53qft8p6c9"
    # 设置 POST 访问
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    # 通过 POST 访问获取账户对应的 access_token
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")
get_access_token()

'24.02338076a792e84ad97734ac6c6b1959.2592000.1712807543.282335-46708986'

In [49]:
import json
import requests
# 一个封装 Wenxin 接口的函数，参数为 Prompt，返回对应结果
def get_completion_weixin(prompt, temperature = 0.95, access_token = ""):
    '''
    prompt: 对应的提示词
    temperature：温度系数
    access_token：已获取到的秘钥
    '''
    url = f"https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions_pro?access_token={access_token}"
    # 配置 POST 参数
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ],
        "temperature" : temperature
    })
    headers = {
        'Content-Type': 'application/json'
    }
    # 发起请求
    response = requests.request("POST", url, headers=headers, data=payload)
    # 返回的是一个 Json 字符串
    js = json.loads(response.text)
    # print(js)
    return js["result"]

In [93]:
prompt_input = '''
请判断以下问题中是否包含对输出的格式要求，并按以下要求输出：
请返回给我一个可解析的python输出列表，列表第一个元素是对输出的格式要求，应该是一个指令；第二个元素是去掉格式要求的问题原文
如果没有格式要求，请将第一个元素置为空.注意，只返回列表，不要markdown格式。
需要判断的问题：
```
{}
```
'''

In [94]:
response = get_completion_weixin(prompt_input.format(question),access_token="24.02338076a792e84ad97734ac6c6b1959.2592000.1712807543.282335-46708986")
print(response)

["返回 Python List", "LLM的分类是什么？"]


In [95]:
prompt_output = '''
请根据回答文本和输出格式要求，按照给定的格式要求对问题做出回答。注意，只返回列表，不要markdown格式。
需要回答的问题：
```
{}
```
回答文本：
```
{}
```
输出格式要求：
```
{}
```
'''

In [96]:
question = 'LLM的分类是什么？给我返回一个 Python List'
# 首先将格式要求与问题拆分
input_lst_s = get_completion_weixin(prompt_input.format(question),access_token="24.02338076a792e84ad97734ac6c6b1959.2592000.1712807543.282335-46708986")
rule, new_question = eval(input_lst_s)


In [97]:
# 接着使用拆分后的问题调用检索链
result = qa_chain({"query": new_question})
result_context = result["result"]
# 接着调用输出格式解析
response = get_completion_weixin(prompt_output.format(new_question, result_context, rule),access_token="24.02338076a792e84ad97734ac6c6b1959.2592000.1712807543.282335-46708986")
print(response)

[INFO] [03-12 14:13:53] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-12 14:13:53] openapi_requestor.py:316 [t:140352797660992]: requesting llm api endpoint: /chat/completions_pro


["基础LLM", "指令微调（Instruction Tuned）LLM"]
